# Loading Dataset
2025.03.25 현민영
- 기간 : 2024-01-01 ~ 2025-02-28
- 종목
    - "삼성전자": "005930.KS",
    - "SK하이닉스": "000660.KS",
    - "네이버": "035420.KS",
    - "셀트리온": "068270.KS",
    - "LG디스플레이": "034220.KS"

1. 주식 데이터
- yfinance의 주가 데이터셋
- stock_data

2. merge_Close
- 5개 종목의 종가 데이터셋

3. 지표 추가된 데이터
- 1에 여러가지 기술적 지표가 추가된 데이터셋
- with_indicators

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

# 금융데이터 관련 라이브러리
import yfinance as yf
import ta                   #금융데이터 기술 분석
import mplfinance as mpf

ModuleNotFoundError: No module named 'ta'

## 1. 주식 데이터
- column (가격+거래량) : 'Close', 'High', 'Low', 'Open', 'Volume'

- row (날짜) : '2024-01-01' ~ '2025-02-28'   
    - 휴장일 데이터 없음.

- csv명 : '삼성전자.csv', 'SK하이닉스.csv', '네이버.csv', '셀트리온.csv,' 'LG디스플레이.csv'

In [3]:
# 주식 데이터 csv 저장 함수
def save_data(tickers, start, end):
    for name, ticker in tickers.items():
        try:
            # 주식 데이터 다운로드
            data = yf.download(ticker, start=start, end=end)
            
            # MultiIndex 제거 (오류 방지를 위한 티커 제거)
            data.columns = data.columns.get_level_values(0)
            
            if not data.empty:
                # CSV 파일로 저장
                data.to_csv(f"stock_data/{name}.csv")
                print(f"{name} 데이터를 저장했습니다.")
            else:
                print(f"{name}에 대한 데이터가 없습니다.")
        except Exception as e:
            print(f"{name} ({ticker}) 데이터 가져오는 중 오류 발생: {e}")


In [4]:
# 관심 종목 리스트
tickers = {
    "SamsungElectronics": "005930.KS",
    "SKHynix": "000660.KS",
    "Naver": "035420.KS",
    "Celltrion": "068270.KS",
    "LGDisplay": "034220.KS"
}

start_date = "2024-01-01"
end_date = "2025-03-01"
save_data(tickers, start_date, end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


SamsungElectronics (005930.KS) 데이터 가져오는 중 오류 발생: Cannot save file into a non-existent directory: 'stock_data'


[*********************100%***********************]  1 of 1 completed


SKHynix (000660.KS) 데이터 가져오는 중 오류 발생: Cannot save file into a non-existent directory: 'stock_data'


...

Naver (035420.KS) 데이터 가져오는 중 오류 발생: Cannot save file into a non-existent directory: 'stock_data'


[*********************100%***********************]  1 of 1 completed


Celltrion (068270.KS) 데이터 가져오는 중 오류 발생: Cannot save file into a non-existent directory: 'stock_data'


[*********************100%***********************]  1 of 1 completed

LGDisplay (034220.KS) 데이터 가져오는 중 오류 발생: Cannot save file into a non-existent directory: 'stock_data'


## 2. merge_Close.csv
yfinance 라이브러리에서 다운로드한 주식 데이터 중에서 Close(종가)만 수집.
- column (주식 종목) :'SamsungElectronics', 'SKHynix', 'Naver', 'Celltrion', 'LG Display'

- row (날짜) : '2024-01-01' ~ '2025-02-28'

In [25]:
# 각 종목별로 CSV 파일 로드
samsung = pd.read_csv("stock_data/SamsungElectronics.csv", index_col=0, parse_dates=True)
sk_hynix = pd.read_csv("stock_data/SKHynix.csv", index_col=0, parse_dates=True)
naver = pd.read_csv("stock_data/Naver.csv", index_col=0, parse_dates=True)
celltrion = pd.read_csv("stock_data/Celltrion.csv", index_col=0, parse_dates=True)
lg_display = pd.read_csv("stock_data/LGDisplay.csv", index_col=0, parse_dates=True)

print("삼성전자 데이터:\n", samsung.head(3), '\n', samsung.tail(3))

삼성전자 데이터:
                    Close          High           Low          Open    Volume
Date                                                                        
2024-01-02  77916.601562  78112.371918  76546.209073  76546.209073  17142847
2024-01-03  75371.585938  77133.519115  75371.585938  76839.863586  21753644
2024-01-04  74980.054688  75665.251010  74490.628743  74490.628743  15324439 
               Close     High      Low     Open    Volume
Date                                                    
2025-02-25  57200.0  57800.0  56500.0  56600.0  14625181
2025-02-26  56600.0  57100.0  56100.0  57000.0  18117091
2025-02-27  56300.0  57100.0  56200.0  56500.0  14975356


In [26]:
# 각 종목의 Close(종가)만 합친 df

# 'Close' 열만 추출하여 각각의 데이터프레임에서 추출
samsung_close = samsung[['Close']]
sk_hynix_close = sk_hynix[['Close']]
naver_close = naver[['Close']]
celltrion_close = celltrion[['Close']]
lg_display_close = lg_display[['Close']]

# 데이터를 종목별로 합쳐 하나의 데이터프레임으로 만듬
merged_df = pd.concat([samsung_close, sk_hynix_close, naver_close, celltrion_close, lg_display_close], axis=1)
merged_df.columns = ['SamsungElectronics', 'SKHynix', 'Naver', 'Celltrion', 'LGDisplay'] # 열 이름 수정
merged_df.head(3)

merged_df.to_csv('stock_data/merge_Close.csv')

## 3. 기술적 지표를 추가한 df
- column (가격+거래량+기술적 지표) : 
    - 주식 가격&거래량 : 'Close', 'High', 'Low', 'Open', 'Volume'
    - 추세 : 'EMA_5', 'EMA_20', 'SMA_5', 'SMA_20', 'MACD', 'MACD_Signal'
    - 변동성 : 'BB_High', 'BB_Low', 'ATR'
    - 모멘텀 : 'RSI', 'Stochastic_K', 'Stochastic_D'
    - 거래량 : 'OBV', 'VWAP'
    - 비율,,,? : 'High_Close_Ratio', 'Low_Close_Ratio', 'Close_Diff_Ratio', 'Close_LastClose_Ratio', 'Volume_LastVolume_Ratio'
    /'Close_MA5_Ratio', 'Close_MA20_Ratio', 'Volume_MA5_Ratio', 'Volume_MA20_Ratio'


- row (날짜) : '2024-01-01' ~ '2025-02-28'   
    - 휴장일 데이터 없음.

In [27]:
# 학습데이터 지표 계산 함수
def calculate_train_df (df):
    """
    입력받은 DataFrame에 주요 기술적 지표를 계산하여 반환하는 함수.
    DataFrame: OHLCV 데이터를 포함하는 DataFrame
    반드시 'Open', 'High', 'Low', 'Close', 'Volume' 열을 포함해야 함.
    """
    # 가격 관련 비율 지표
    df['Open_Close_Ratio'] = df['Open'] / df['Close'].shift(1)
    df['High_Close_Ratio'] = df['High'] / df['Close']
    df['Low_Close_Ratio'] = df['Low'] / df['Close']
    df['Close_Diff_Ratio'] = df['Close'].pct_change()
    df['Close_LastClose_Ratio'] = df['Close'] / df['Close'].shift(1)
    df['Volume_LastVolume_Ratio'] = df['Volume'] / df['Volume'].shift(1)

    # 가격 이동평균 비율
    df['Close_MA5_Ratio'] = df['Close'] / df['Close'].rolling(window=5).mean()
    df['Close_MA20_Ratio'] = df['Close'] / df['Close'].rolling(window=20).mean()

    # 거래량 이동평균 비율
    df['Volume_MA5_Ratio'] = df['Volume'] / df['Volume'].rolling(window=5).mean()
    df['Volume_MA20_Ratio'] = df['Volume'] / df['Volume'].rolling(window=20).mean()

     # 이동평균 지표
    df['EMA_5'] = ta.trend.ema_indicator(close=df['Close'], window=5)
    df['EMA_20'] = ta.trend.ema_indicator(close=df['Close'], window=20)
    df['SMA_5'] = ta.trend.sma_indicator(close=df['Close'], window=5)
    df['SMA_20'] = ta.trend.sma_indicator(close=df['Close'], window=20)

    # MACD
    macd = ta.trend.MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()

    # Bollinger Bands
    bb = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2)
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Low'] = bb.bollinger_lband()

    # ATR (Average True Range)
    df['ATR'] = ta.volatility.average_true_range(
        high=df['High'], low=df['Low'], close=df['Close'], window=14
    )

    # RSI (Relative Strength Index)
    df['RSI'] = ta.momentum.rsi(close=df['Close'], window=14)

    # Stochastic Oscillator
    stoch_k = ta.momentum.stoch(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3)
    stoch_d = ta.momentum.stoch_signal(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3)
    df['Stochastic_K'] = stoch_k
    df['Stochastic_D'] = stoch_d

    # OBV (On-Balance Volume)
    df['OBV'] = ta.volume.on_balance_volume(close=df['Close'], volume=df['Volume'])

    # VWAP (Volume Weighted Average Price)
    df['VWAP'] = ta.volume.volume_weighted_average_price(
        high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume']
    )
    
    print(df.info())
    return df

In [28]:
a = calculate_train_df(samsung)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 281 entries, 2024-01-02 to 2025-02-27
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Close                    281 non-null    float64
 1   High                     281 non-null    float64
 2   Low                      281 non-null    float64
 3   Open                     281 non-null    float64
 4   Volume                   281 non-null    int64  
 5   Open_Close_Ratio         280 non-null    float64
 6   High_Close_Ratio         281 non-null    float64
 7   Low_Close_Ratio          281 non-null    float64
 8   Close_Diff_Ratio         280 non-null    float64
 9   Close_LastClose_Ratio    280 non-null    float64
 10  Volume_LastVolume_Ratio  280 non-null    float64
 11  Close_MA5_Ratio          277 non-null    float64
 12  Close_MA20_Ratio         262 non-null    float64
 13  Volume_MA5_Ratio         277 non-null    float64
 14  Volume_

In [29]:
samsung.isna().sum().sort_values().tail(1)
# MACD Signal의 결측행 수가 최대 (33행) -> 앞의 33일 데이터가 필요

MACD_Signal    33
dtype: int64

In [30]:
# 주식데이터 로드 함수
def download_data (ticker, start, end):
    df = yf.download(ticker, start, end)

    # MultiIndex 제거 (오류 방지를 위한 티커 제거)
    df.columns = df.columns.get_level_values(0)

    print(df.shape)
    
    return df


# 5개의 주식 종목 다운로드
# 2024-01-01보다 앞 33일 데이터 다운로드
print('stock_data info\n ============================================')
samsung2 = download_data('005930.KS','2023-11-11', '2025-02-28')
sk_hynix2 = download_data('000660.KS','2023-11-11', '2025-02-28')
naver2 = download_data('000660.KS','2023-11-11', '2025-02-28')
celltrion2 = download_data('000660.KS','2023-11-11', '2025-02-28')
lg_display2 = download_data('000660.KS','2023-11-11', '2025-02-28')

# 기술적 지표 계산
calculate_train_df(samsung2)
calculate_train_df(sk_hynix2)
calculate_train_df(naver2)
calculate_train_df(celltrion2)
calculate_train_df(lg_display2)

# 결측치 제거
samsung2.dropna(inplace=True)
sk_hynix2.dropna(inplace=True)
naver2.dropna(inplace=True)
celltrion2.dropna(inplace=True)
lg_display2.dropna(inplace=True)

# 확인
print(naver2[naver2.isna().any(axis=True)])
print(naver2.head(2))
print(naver2.tail(2))

[*********************100%***********************]  1 of 1 completed

stock_data info
(314, 5)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


(314, 5)
(314, 5)
(314, 5)
(314, 5)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 314 entries, 2023-11-13 to 2025-02-27
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Close                    314 non-null    float64
 1   High                     314 non-null    float64
 2   Low                      314 non-null    float64
 3   Open                     314 non-null    float64
 4   Volume                   314 non-null    int64  
 5   Open_Close_Ratio         313 non-null    float64
 6   High_Close_Ratio         314 non-null    float64
 7   Low_Close_Ratio          314 non-null    float64
 8   Close_Diff_Ratio         313 non-null    float64
 9   Close_LastClose_Ratio    313 non-null    float64
 10  Volume_LastVolume_Ratio  313 non-null    float64
 11  Close_MA5_Ratio          310 non-null    float64
 12  Close_MA20_Ratio         295 non-null    float64
 13  Volume_MA5_Ratio         

In [31]:
# csv로 저장
samsung2.to_csv("with_indicators_data/SamsungElectronics_indicators.csv")
sk_hynix2.to_csv("with_indicators_data/SKHynix_indicators.csv")
naver2.to_csv("with_indicators_data/Naver_indicators.csv")
celltrion2.to_csv("with_indicators_data/Celltrion_indicators.csv")
lg_display2.to_csv("with_indicators_data/LGDisplay_indicators.csv")